## Form vocabulary from FastText pretrained vectors and our train data


In [19]:
import itertools
import nltk
#from keras.preprocessing.sequence import pad_sequences
from utilities import *
import re
import io
import random
import matplotlib as plt
import matplotlib.pyplot as plt

In [20]:
# File paths
txt_file_path = "../../final_all_GS_tagged_words_symbols_sentences.txt"
txt_file_path0 = "../../final_all_GS_tagged_words_symbols_sentences_0.txt"
txt_file_path00 = "../../final_all_GS_tagged_words_symbols_sentences_00.txt"
txt_file_path1 = "../../korr_final_symbols_sentences.txt"
txt_file_path2 = "../../ukrlib_final_symbols_sentences.txt" 
txt_file_path10 = "../../korr_final_symbols_sentences_01.txt"
txt_file_path20 = "../../ukrlib_final_symbols_sentences_01.txt"
txt_file_path100 = "../../korr_final_symbols_sentences_00.txt"
txt_file_path200 = "../../ukrlib_final_symbols_sentences_00.txt"

txt_file_path300 = "../../korr_ukrlib_final_symbols_sentences_00.txt"


data_path = "data/korr_ukrlib_data_fast.pkl"
test_path = "data/brown_test_data_fast.pkl"

# Sentence tokens
unknown_token = "_#unknown_"
sentence_start_token = "_#start_"
sentence_end_token = "_#end_"
padding_token = "_#padding_"

#Number of words to hold in vocabulary
vocabulary_size = 300000

In [21]:
print("Loading data ...")
data = load_training_data(data_path)
data_test = load_training_data(test_path)
print("DONE loading data ...")

Loading data ...
Loaded file training data from data/korr_ukrlib_data_fast.pkl.
Loaded file training data from data/brown_test_data_fast.pkl.
DONE loading data ...


In [22]:
len(data['index_to_fast'])

300000

In [23]:
max([max(sen) for sen in data['x_train']])

299999

In [24]:
min([min(sen) for sen in data['x_train']])

0

In [3]:
with open(txt_file_path2, "rt") as infile:
    with open(txt_file_path20, "w+") as outfile:
        for sent in infile:
            if len(sent.split())>58:
                sep=';'
                separ='////'
                for k in sent.replace(sep, sep+separ).split(separ):
                    print(re.sub( r'^   |\n|^\t','',k), file=outfile)
            else:
                outfile.write(sent)
                
with open(txt_file_path1, "rt") as infile:
    with open(txt_file_path10, "w+") as outfile:
        for sent in infile:
            if len(sent.split())>58:
                sep=';'
                separ='////'
                for k in sent.replace(sep, sep+separ).split(separ):
                    print(re.sub( r'^   |\n|^\t','',k), file=outfile)
            else:
                outfile.write(sent)
with open(txt_file_path, "rt") as infile:
    with open(txt_file_path0, "w+") as outfile:
        for sent in infile:
            if len(sent.split())>58:
                sep=';'
                separ='////'
                for k in sent.replace(sep, sep+separ).split(separ):
                    print(re.sub( r'^   |\n|^\t','',k), file=outfile)
            else:
                outfile.write(sent)

In [3]:
with open(txt_file_path300, "rt") as infile:
    tokenized_sentences = [("%s %s %s" % (sentence_start_token, sentence, sentence_end_token)).split() for sentence in infile]

In [4]:
with open(txt_file_path00, "rt") as infile:
    tokenized_sentences_test = [("%s %s %s" % (sentence_start_token, sentence, sentence_end_token)).split() for sentence in infile]

#### Train corpus sentences<60

In [5]:
print ("Number of sentences ", len(tokenized_sentences))
print("Number of all tokens (with _#start_ _#end_ in each sentence) ", sum([len(sen) for sen in tokenized_sentences]) )
print("Number of all tokens (without _#start_ _#end_ in each sentence) ",  sum([len(sen)-2 for sen in tokenized_sentences])) 

Number of sentences  14335495
Number of all tokens (with _#start_ _#end_ in each sentence)  262598163
Number of all tokens (without _#start_ _#end_ in each sentence)  233927173


#### Test corpus sentences <60

In [6]:
print ("Number of sentences ", len(tokenized_sentences_test))
print("Number of all tokens (with _#start_ _#end_ in each sentence) ", sum([len(sen) for sen in tokenized_sentences_test]) )
print("Number of all tokens (without _#start_ _#end_ in each sentence) ",  sum([len(sen)-2 for sen in tokenized_sentences_test])) 

Number of sentences  39900
Number of all tokens (with _#start_ _#end_ in each sentence)  779001
Number of all tokens (without _#start_ _#end_ in each sentence)  699201


#### Example of the longest sentence

In [7]:
print(max(tokenized_sentences, key=len))

['_#start_', '"', 'Одеська', '"', 'вирячила', 'здуру', 'на', 'молодика', ',', 'що', 'зазирав', 'одним', 'рогом', 'у', 'вікно', ',', 'свої', 'сіро-зелені', 'баньки', ',', 'раптом', ',', 'немов', 'читаючи', 'на', 'небі', 'ноти', ',', 'заверещала', 'тоненьким', 'голоском', ',', 'наче', 'сука', 'на', 'місяць', ':', 'Спи', 'же', ',', 'спи', ',', 'моя', 'родная', ',', 'Бог', 'твой', 'сон', 'хранит', ',', 'Твоя', 'мама', 'шансонетка', ',', 'По', 'ночам', 'не', 'спит', '...', '_#end_']


#### Count the word frequencies train

In [8]:
word_frequency = nltk.FreqDist(itertools.chain(*tokenized_sentences)) 

In [9]:
# Get the most common words and build index to word and word to index vectors
vocab = word_frequency.most_common(len(word_frequency.items()))
print("Found %d unique word tokens." % len(word_frequency.items()))

Found 2189477 unique word tokens.


#### Count the word frequencies test

In [10]:
word_frequency_test = nltk.FreqDist(itertools.chain(*tokenized_sentences_test)) 

In [11]:
type(word_frequency_test)

nltk.probability.FreqDist

In [12]:
# Get the most common words and build index to word and word to index vectors
vocab_test = word_frequency_test.most_common(len(word_frequency_test.items()))
print("Found %d unique word tokens." % len(word_frequency_test.items()))

Found 100420 unique word tokens.


#### Vocabulary fast

In [13]:
vocab_train_fast=[]
vocabkeys = [pair[0].replace("'",'') for pair in vocab]
vocabkeys2 = [pair[0] for pair in vocab]

vocabkeys_set = set(vocabkeys)
vocabkeys2_set = set(vocabkeys2)
vocabkeys_map = dict()
for step, vocab_key in enumerate(vocabkeys):
    vocabkeys_map[vocab_key] = step
vocab_dict=dict(vocab)

In [14]:
present=[]
absent=[]
fin = io.open('../../cc.uk.300.vec', 'r', encoding='utf-8', newline='\n', errors='ignore')
for line in fin:
        tokens = line.rstrip().split(' ')
        if tokens[0] in vocabkeys2_set:
            vocab_train_fast.append((tokens[0],vocab_dict[tokens[0]],list(map(float, tokens[1:]))))
            present.append(tokens[0])
        elif tokens[0] in vocabkeys_set:
            vocab_train_fast.append((vocabkeys2[vocabkeys_map[tokens[0]]],vocab_dict[vocabkeys2[vocabkeys_map[tokens[0]]]],list(map(float, tokens[1:]))))
            present.append(tokens[0])
        else:
            absent.append(tokens[0])
        #vocab.append(tokens[0])
    #return vocab, np.stack(weights)

In [15]:
absent_in_fast=vocabkeys2_set-set(present)-set([sentence_end_token, sentence_start_token])
unk_freak=sum([vocab_dict[v] for v in absent_in_fast])

In [16]:
unk_freak

10335097

In [17]:
len(vocab_train_fast)

763578

In [18]:
i=-1
fin = io.open('../../cc.uk.300.vec', 'r', encoding='utf-8', newline='\n', errors='ignore')
for line in fin:
    i+=1
    #print('yes')
    tokens = line.rstrip().split(' ')
    #data[tokens[0]] = list(map(float, tokens[1:]))
    if i==1:
        unk=np.array(list(map(float, tokens[1:])))
    elif i>1:
        unk+=np.array(list(map(float, tokens[1:])))
        unk/2
#vec2=vect/2

In [20]:
i=0
r=0
fin = io.open('../../cc.uk.300.vec', 'r', encoding='utf-8', newline='\n', errors='ignore')
for line in fin:
    r+=1
    #print('yes')
    tokens = line.rstrip().split(' ')
    #data[tokens[0]] = list(map(float, tokens[1:]))
    if i==0 and tokens[0].isdigit() and r>1:
        i+=1
        numbers=np.array(list(map(float, tokens[1:])))
    elif i>0 and tokens[0].isdigit():
        numbers+=np.array(list(map(float, tokens[1:])))
        numbers/2
#vec2=vect/2

In [21]:
vocab_train_fast.append(('_#number_',vocab_dict['_#number_'],list(numbers)))
vocab_train_fast.append(('_#unknown_',unk_freak,list(unk)))
vocab_train_fast.append((sentence_end_token,vocab_dict[sentence_end_token],[random.uniform(-1,1) for i in range(300)]))
vocab_train_fast.append((sentence_start_token,vocab_dict[sentence_start_token],[random.uniform(-1,1) for i in range(300)]))

In [22]:
vocab_train_fast.sort(key=lambda tup: tup[1], reverse = True)

In [23]:
len(vocab_train_fast)

763582

In [24]:
vocab_dict['_#number_']

2182152

#### Train vocabulary 300000

In [26]:
vocabulary = vocab_train_fast[:300000]
len(vocabulary)

300000

In [27]:
vocabulary[4]

('_#unknown_',
 10335097,
 [42152.47709999825,
  -2113.3825999999785,
  1532.3419000000683,
  39876.290299997694,
  -11239.409200000297,
  -3118.373800000167,
  -12070.744099999265,
  -31521.033399998738,
  -33498.35169999813,
  -60447.93899999902,
  47054.68559999562,
  1777.295400000074,
  -38.067699999966635,
  -45705.3566999965,
  18276.20599999667,
  3552.121100000272,
  -741.4504999999903,
  -7665.517900000761,
  4476.051000000697,
  44728.22319999629,
  17941.48070000008,
  -55487.12080000034,
  13841.112899999493,
  1755.1929000001148,
  -7676.18930000046,
  10327.149999999703,
  -7216.319699997982,
  26088.255399996575,
  1983.6393999999063,
  -2588.206700000004,
  -7571.823200001247,
  1787.8362999999604,
  4.570400000022857,
  -1279.525899999928,
  10869.424400000717,
  5077.936100000569,
  -780.6250000000095,
  26614.84829999766,
  11797.106799999781,
  22085.686299997047,
  -18963.91879999885,
  -3651.871100000314,
  223.08250000001686,
  3049.9829999999347,
  2818.0362999

In [28]:
# Generate word to index and index to words (Add the word not the frequency from our vocabulary data)
index_to_word = [x[0] for x in vocabulary]
#index_to_word.insert(0, unknown_token)
#index_to_word.insert(0, padding_token)

In [29]:
index_to_word[0]
#index_to_word[1]

','

In [30]:
# Dictionary of {word : index} pairs
word_to_index = dict([(word, i) for i, word in enumerate(index_to_word)])

In [31]:
index_to_fast=np.stack([x[2] for x in vocabulary])

#### Replace all words not in our vocabulary with the #unknown token

In [32]:
unknown=0
# Replace all words not in our vocabulary with the unknown token
for i, sentence in enumerate(tokenized_sentences):
    for j, word in enumerate(sentence):
        if word not in word_to_index:
            tokenized_sentences[i][j] = unknown_token
            unknown+=1

In [33]:
unknown

9820363

In [34]:
unknown_test=0
# RepB8bvjbx3mUQXnxB8bvjbx3mUQXnxlace all words not in our vocabulary with the unknown token
for i, sentence in enumerate(tokenized_sentences_test):
    for j, word in enumerate(sentence):
        if word not in word_to_index:
            tokenized_sentences_test[i][j] = unknown_token
            unknown_test+=1

In [35]:
unknown_test

35187

In [36]:
vocabulary[4]=(vocabulary[4][0], unknown, vocabulary[4][2])
#vocabulary.append((padding_token, 0))

In [38]:
#print("Using vocabulary size %d." % vocabulary_size)
print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (
    vocabulary[-2][0], vocabulary[-2][1]))
print("Example sentence after Pre-processing: '%s'" % tokenized_sentences[101265])

The least frequent word in our vocabulary is 'Телевізійну' and appeared 14 times.
Example sentence after Pre-processing: '['_#start_', '—', 'Ти', 'завтра', 'їдеш', 'на', 'озеро', '?', '—', 'запитала', 'вже', 'зовсім', 'приязно', '.', '_#end_']'


#### Form train data file

In [39]:
x_train = []
for sentence in tokenized_sentences:
    x = []
    for word in sentence:
        x.append(word_to_index[word])
    x_train.append(x)

In [40]:
random.shuffle(x_train)
len(x_train)

14335495

In [41]:
# Save data to file
data = dict(
    x_train=x_train,
    #y_train=y_train,
    word_to_index=word_to_index,
    index_to_word=index_to_word,
    index_to_fast=index_to_fast,
    num_sentences=len(x_train),
    max_input_len=len(max(x_train,key=len)))

In [42]:
print("Saving training data")
try:
    save_training_data(data_path, data)
except FileNotFoundError as err:
    print("Error saving data " + str(err))

Saving training data
Saved file training data to data/korr_ukrlib_data_fast.pkl.


In [46]:
len(index_to_fast)

300000

#### Form test data file

In [43]:
x_test = []
for sentence in tokenized_sentences_test:
    x = []
    for word in sentence:
        x.append(word_to_index[word])
    x_test.append(x)

In [44]:
random.shuffle(x_test)
test_data = dict(
    x_test=x_test)

In [45]:
print("Saving test data")
try:
    save_training_data(test_path, test_data)
except FileNotFoundError as err:
    print("Error saving data " + str(err))

Saving test data
Saved file training data to data/brown_test_data_fast.pkl.


#### Chunks

In [47]:
"""
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

x_train_new=[]
y_train_new=[]
for somelist in x_train:
    if len(somelist)>100:
        somelist=list(chunks(somelist,100))
        for nested in somelist:
            x_train_new.append(nested)
    else:
        x_train_new.append(somelist)
for somelist in y_train:
    if len(somelist)>100:
        somelist=list(chunks(somelist,100))
        for nested in somelist:
            y_train_new.append(nested)
    else:
        y_train_new.append(somelist) """      

'\ndef chunks(l, n):\n    # For item i in a range that is a length of l,\n    for i in range(0, len(l), n):\n        # Create an index range for l of n items:\n        yield l[i:i+n]\n\nx_train_new=[]\ny_train_new=[]\nfor somelist in x_train:\n    if len(somelist)>100:\n        somelist=list(chunks(somelist,100))\n        for nested in somelist:\n            x_train_new.append(nested)\n    else:\n        x_train_new.append(somelist)\nfor somelist in y_train:\n    if len(somelist)>100:\n        somelist=list(chunks(somelist,100))\n        for nested in somelist:\n            y_train_new.append(nested)\n    else:\n        y_train_new.append(somelist) '